In [5]:
%serialconnect

Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.


In [11]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
pinfan       12
boardname    esp8266sens2
wifialt      DoESLiverpool,decafbad00,10.0.100.1

Sent 7 lines (173 bytes) to config.txt.


In [7]:
%sendtofile --source ../Sensor_Kennel/BME280_funcs.py

Sent 79 lines (3304 bytes) to BME280_funcs.py.


In [18]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 33 lines (1137 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [28]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB8 --baud=115200 
Ready.


In [29]:
%sendtofile main.py

import time, itertools
from machine import Pin
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, pinled
from mqtt_as import MQTTClient
from BME280_funcs import bme280init, readBME280

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
print(list(map(hex, i2c.scan())))

boardname = fconfig["boardname"].encode()    
topicbme280A = boardname+b'/bme280/A'
topicbme280B = boardname+b'/bme280/B'
pinfan = Pin(int(fconfig["pinfan"]), Pin.OUT)
topicfanon = boardname+b'/fan/on'
topicstatus = boardname+b"/ip"

# dereferenced so we can swap the sample rate so we can 
# see how it affects the temperature and fan combinations
bmeparams = [[0x76, boardname+b'/bme280/A', 500],
             [0x77, boardname+b'/bme280/B', 60000]]

async def fantask():
    pinfan.value(0)
    while not client.isconnected():
        await asyncio.sleep_ms(1000)
    for i in itertools.count():
        if (i%3) == 2:
            bmeparams[0][2], bmeparams[1][2] = bmeparams[1][2], bmeparams[0][2]
        if (i%4) == 0:
            v = 1-pinfan.value()
            print(topicfanon, v)
            await client.publish(topicfanon, str(v))
            pinfan.value(v)
        await asyncio.sleep_ms(15*60*1000)
    
async def bme280task(k):
    btoinit = True
    addr, topic = bmeparams[k][:2]
    for i in itertools.count():
        try:
            if btoinit:
                bme280init(i2c, addr)
                btoinit = False
                await asyncio.sleep_ms(100)
            meas = readBME280(addr)
            payload = " ".join(map(str, meas))
            if client.isconnected():
                print(topic, meas)
                await client.publish(topic, payload)
        except OSError as e:
            print("bme280task", e)
            btoinit = True
        if bmeparams[k][2] < 1000:
            pinled.value((i%10) < (1 if client.isconnected() else 5))
        await asyncio.sleep_ms(bmeparams[k][2])
        
async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topicstatus, ipnumber, retain=True)
    
config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(bme280task(0))
aloop.create_task(bme280task(1))
aloop.create_task(fantask())
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()



Sent 73 lines (2439 bytes) to main.py.


In [10]:
print(fconfig)

{'boardname': 'esp8266sens4', 'pinpir': '13', 'mqttbroker': '10.0.100.1', 'wifiname': 'DoESLiverpool', 'wifipassword': 'decafbad00', 'wifialt': 'DoESLiverpool,decafbad00,10.0.100.1', 'pinled': '2'}


In [ ]:
async def callbackcmdtask(topic, msg):
    print("executing", [msg])
    pinled.value(0)
    try:
        exec(msg)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(1)
    return
    
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(topic, msg))


In [22]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [20]:
# below is no-sensor running to find why it keeps timing out

In [4]:
%sendtofile main.py

import time
from machine import Pin
from mqtt_as import config, MQTTClient
import uasyncio as asyncio

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)
    
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
boardname = fconfig["boardname"]

async def ledtask():
    while True:
        pinled.value(0)
        await asyncio.sleep_ms(20 if client.isconnected() else 1600)
        pinled.value(1)
        await asyncio.sleep_ms(2000)
        
async def sendnumbertask():
    i = 0
    topic = boardname + "/number"
    while True:
        if client.isconnected():
            print(i)
            await client.publish(topic, str(i))
        await asyncio.sleep_ms(1000)
        i += 1

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+'/ip', ipnumber, retain=True)
    
async def mqtttask(bflip=False):
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        print("connecting to:", client._ssid)
        await client.connect()
        return
    except OSError as e:
        print("client connect", [e])
    aloop.create_task(mqtttask(True))

config['connect_coro'] = onconnecttask
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(sendnumbertask())
aloop.create_task(ledtask())
aloop.create_task(mqtttask())
aloop.run_forever()



Sent 62 lines (1878 bytes) to main.py.


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [6]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-576-gd667bc642 on 2019-11-12; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f24e5e062b0, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [4]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [5]:
import machine
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))


In [6]:
print(i2c.scan())

[57]
